## Deployment Azure

#### Import Libraries

In [1]:
import datetime
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.externals import joblib
 
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies
import warnings
warnings.filterwarnings("ignore")

C:\Users\Darwoft\Anaconda3\envs\aml\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


#### Deployment

In [2]:
#Check Azure ML SDK version
print(azureml.core.VERSION)

1.0.39


In [3]:
#If you want to create a Azure ML workspace by code

"""ws = Workspace.create(name='salary',
                      subscription_id='', 
                      resource_group='mi2',
                      create_resource_group=True,
                      location='southeastasia'
                     )
"""

#If you have already created a Azure ML Workspace and you want to work with it
ws= Workspace(subscription_id='2c551064-b082-40a9-b869-5eb2ed9de3c0', 
          resource_group='SentimentAnalysis', workspace_name='sentiment', auth=None
          , _location='southcentralus', _disable_service_check=False)

In [4]:
#Write configuration to local file
ws.write_config()

In [5]:
#Load and Test the models
filename = 'outputs/topicclassifier.pkl'
loaded_model=joblib.load(filename)
filename = 'outputs/vectorizer.pkl'
vectorizer=joblib.load(filename)
y=loaded_model.predict(vectorizer.transform(["Nuestro #Madretón continúa lleno de ofertas y especiales ¡Te esperamos! "]))
print(y)

['ofertas']


In [6]:
# Registering and Serving the Trained Model
model1 = Model.register(model_path = "outputs/topicclassifier.pkl",
                       model_name = "topicclassifier",
                       tags = {"key": "1"},
                       description = "topicclassifier",
                       workspace = ws)
model2 = Model.register(model_path = "outputs/vectorizer.pkl",
                       model_name = "vectorizer",
                       tags = {"key": "1"},
                       description = "vectorizer",
                       workspace = ws)

Registering model topicclassifier
Registering model vectorizer


In [7]:
#Define Azure ML Deploymemt configuration
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "topic",  "method" : "sklearn"}, 
                                               description='Topic Classifier')

In [8]:
#Create enviroment configuration file
topicclassifierenv = CondaDependencies()
topicclassifierenv.add_conda_package("scikit-learn")
 
with open("topicclassifierenv.yml","w") as f:
    f.write(topicclassifierenv.serialize_to_string())
with open("topicclassifierenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will

# be automatically provisioned for runs with userManagedDependencies=False.


# Details about the Conda environment file format:

# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually


name: project_environment
dependencies:
  # The python interpreter version.

  # Currently Azure ML only supports 3.5.2 and later.

- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.

  - azureml-defaults
- scikit-learn



In [9]:
#creates a file called score.py that contains the inference logic for the model
%%writefile score.py
 
import json
import numpy as np
import os
import pickle
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
 
from azureml.core.model import Model
 
def init():
    global topicclassifier,vectorizer
    # retrieve the path to the model file using the model name
    model_path1 = Model.get_model_path('vectorizer')
    model_path2 = Model.get_model_path('topicclassifier')
    vectorizer = joblib.load(model_path1)
    topicclassifier = joblib.load(model_path2)
 
def run(raw_data):
    data =  np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = topicclassifier.predict(vectorizer.transform(data))
    return json.dumps(y_hat.tolist())

Overwriting score.py


In [10]:
#Deploy the model to Azure Container Instance
%%time
 
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="topicclassifierenv.yml")

Wall time: 2.86 ms


In [11]:
#Expose web service
service = Webservice.deploy_from_model(workspace=ws,
                                       name='topicclassifierwebservice',
                                       deployment_config=aciconfig,
                                       models=[model1,model2],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

Creating image

Image creation operation finished for image topicclassifierwebservice:2, operation "Succeeded"
Creating service
Running...................
SucceededACI service creation operation finished, operation "Succeeded"


In [12]:
#Get the Web Service URL
print(service.scoring_uri)

http://410ab97b-f7d5-4ed7-8198-e4cbe5b93ef5.westus.azurecontainer.io/score


In [13]:
#Check the logging of the service 
print(service.get_logs())

2019-05-31T14:28:44,212347278+00:00 - nginx/run 
2019-05-31T14:28:44,214304299+00:00 - rsyslog/run 
2019-05-31T14:28:44,213708393+00:00 - iot-server/run 
2019-05-31T14:28:44,218089239+00:00 - gunicorn/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2019-05-31T14:28:44,435933547+00:00 - iot-server/finish 1 0
2019-05-31T14:28:44,445037643+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 19.6.0
Listening at: http://127.0.0.1:9090 (15)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 46
Initializing logger
Starting up app insights client
Starting up request id generator
Starting up app insight hooks
Invoking user's init function
2019-05-31 14:28:48,199 | azureml.core.run | DEBUG | Could not load run context Could not load a submitted run, if outside of an execution context, use experiment.start_logging to initialize an azureml.core.Run., switching offline: False
2019-05-31 14:28:48,199 | azureml.core.run | D

In [15]:
#To make a request to the RESTful web service

import requests
import json

# URL for the web service
scoring_uri ="http://410ab97b-f7d5-4ed7-8198-e4cbe5b93ef5.westus.azurecontainer.io/score"

# If the service is authenticated, set the key
#key = 'salarykeyvault806708ce3c'

data = {"data": 
            [
                "Nuestro #Madretón continúa lleno de ofertas y especiales ¡Te esperamos! "
            ]
        }

# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = { 'Content-Type':'application/json' }

# If authentication is enabled, set the authorization header
#headers['Authorization']=f'Bearer {key}'
# Make the request and display the response

resp = requests.post(scoring_uri, input_data,headers=headers)
print(resp.text)


"[\"ofertas\"]"
